In [1]:
!pip install qiskit_aer
!pip install qiskit_ibm_runtime
from qiskit.visualization import *
from qiskit_ibm_runtime import *
import numpy as np
from qiskit import *
from qiskit_aer import *
from scipy.fft import fft
from scipy.stats import entropy

# Step 1: Generate QRNG data using Qiskit
def generate_qrng_data(num_bits):
    circuit = QuantumCircuit(1, 1)
    circuit.h(0)  # Apply Hadamard gate to create superposition
    circuit.measure(0, 0)  # Measure the qubit

    backend_sim = AerSimulator()  # Initialize the AerSimulator
    compiled_sim = transpile(circuit, backend_sim)  # Transpile the circuit
    result_sim = backend_sim.run(compiled_sim, shots=num_bits).result()  # Run the simulation
    counts = result_sim.get_counts()

    # Convert measurement results to binary bitstring
    bits = []
    for bit, count in counts.items():
        bits.extend([int(bit)] * count)
    return np.array(bits[:num_bits])

# Step 2: Create a Toeplitz matrix
def create_toeplitz(first_row, first_column):
    n = len(first_row)
    m = len(first_column)
    toeplitz_matrix = np.zeros((n, m), dtype=int)
    for i in range(n):
        for j in range(m):
            if j >= i:
                toeplitz_matrix[i, j] = first_row[j - i]
            else:
                toeplitz_matrix[i, j] = first_column[i - j]
    return toeplitz_matrix

# Step 3: Calculate Shannon entropy
def calculate_entropy(data):
    _, counts = np.unique(data, return_counts=True)
    probabilities = counts / len(data)
    return entropy(probabilities, base=2)

# Step 4: Apply Toeplitz transformation dynamically
def apply_toeplitz_dynamically(qrng_data, toeplitz_matrix, block_size):
    transformed_data = []
    for i in range(0, len(qrng_data) - block_size + 1, block_size):
        block = qrng_data[i:i + block_size]
        transformed_block = np.dot(toeplitz_matrix, block) % 2
        transformed_data.extend(transformed_block)
    return np.array(transformed_data)

# Step 5: Apply FFT-based Toeplitz transformation
def apply_fft_toeplitz(qrng_data, toeplitz_matrix, block_size):
    transformed_data = []
    for i in range(0, len(qrng_data) - block_size + 1, block_size):
        block = qrng_data[i:i + block_size]
        fft_block = fft(block)
        fft_toeplitz = fft(toeplitz_matrix, axis=1)
        transformed_block = np.real(np.fft.ifft(fft_block * fft_toeplitz)).astype(int) % 2
        transformed_data.extend(transformed_block)
    return np.array(transformed_data)

# Main Program
if __name__ == "__main__":
    # Parameters
    num_bits = 1000
    block_size = 128

    # Step 1: Generate QRNG data
    qrng_data = generate_qrng_data(num_bits)

    # Step 2: Define Toeplitz matrix
    first_row = np.random.randint(0, 2, block_size)
    first_column = np.random.randint(0, 2, block_size)
    toeplitz_matrix = create_toeplitz(first_row, first_column)

    # Step 3: Apply Toeplitz transformation dynamically
    transformed_data = apply_toeplitz_dynamically(qrng_data, toeplitz_matrix, block_size)

    # Step 4: Apply FFT-based Toeplitz transformation dynamically
    fft_transformed_data = apply_fft_toeplitz(qrng_data, toeplitz_matrix, block_size)

    # Step 5: Calculate Shannon entropy
    original_entropy = calculate_entropy(qrng_data)
    transformed_entropy = calculate_entropy(transformed_data)
    fft_transformed_entropy = calculate_entropy(fft_transformed_data)

    # Step 6: Print Results
    print(f"Original Shannon Entropy: {original_entropy:.4f}")
    print(f"Transformed Shannon Entropy (Toeplitz): {transformed_entropy:.4f}")
    print(f"Transformed Shannon Entropy (FFT Toeplitz): {fft_transformed_entropy:.4f}")

  Using cached qiskit_aer-0.15.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.0 kB)
  Using cached qiskit-1.3.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached rustworkx-0.15.1-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.9 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached stevedore-5.4.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached symengine-0.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached pbr-6.1.0-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached qiskit_aer-0.15.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
Using cached qiskit-1.3.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 110.3 MB